## 1. [50 pts] In this assignment, we will use Apriori analysis to find phrases, or interesting patterns in a novel.
Use the nltk library corpus gutenberg API and load the novel 'carroll-alice.txt' which is the
Alice in Wonderland by L. Carroll.<br> There are 1703 sentences in the novel which can be
represented as 1703 transactions.<br> Use any means to parse/extract words and save in CSV
format to be read by Weka framework similar to the Apriori Analysis module.<br>
Hint: Removing stop words and using regular expressions can be helpful: <br>
from nltk.corpus import gutenberg, stopwords <br>
Stop_words = stopwords.words('english') <br>
Sentences = gutenberg.sents('carroll-alice.txt') <br>
TermsSentences = []<br>
for terms in Sentences: <br>
 terms = [w for w in terms if w not in Stop_words]<br>
 terms = [w for w in terms if re.search(r'^[a-zA-Z]{2}', w) is not None]<br>
Use FPGrowth and start with default parameters. Reduce lowerBoundMinSupport to reach
to a sweet point for the support and avoid exploding the number of rules generated. <br>
Report interesting patterns.<br>
(Example: Some of the frequently occurring phrases are Mock Turtle, White Rabbit, etc.)

In [1]:
import re
import os
import sys
import nltk
import struct
import warnings
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

In [2]:
nltk.download('gutenberg')
nltk.download('stopwords')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/ianzimmer/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ianzimmer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
Stop_words = nltk.corpus.stopwords.words('english')
Sentences = nltk.corpus.gutenberg.sents('carroll-alice.txt')
TermSentences = []
maxi = 0
for terms in Sentences:
    terms = [w for w in terms if w not in Stop_words]
    terms = [w for w in terms if re.search(r'^[a-zA-Z]{2}', w) is not None]
    if len(terms) > maxi:
        maxi = len(terms)
    TermSentences.append(terms)
print(maxi)
print(len(TermSentences))
TermSentences

85
1703


[['Alice', 'Adventures', 'Wonderland', 'Lewis', 'Carroll'],
 ['CHAPTER'],
 ['Down', 'Rabbit', 'Hole'],
 ['Alice',
  'beginning',
  'get',
  'tired',
  'sitting',
  'sister',
  'bank',
  'nothing',
  'twice',
  'peeped',
  'book',
  'sister',
  'reading',
  'pictures',
  'conversations',
  'use',
  'book',
  'thought',
  'Alice',
  'without',
  'pictures',
  'conversation'],
 ['So',
  'considering',
  'mind',
  'well',
  'could',
  'hot',
  'day',
  'made',
  'feel',
  'sleepy',
  'stupid',
  'whether',
  'pleasure',
  'making',
  'daisy',
  'chain',
  'would',
  'worth',
  'trouble',
  'getting',
  'picking',
  'daisies',
  'suddenly',
  'White',
  'Rabbit',
  'pink',
  'eyes',
  'ran',
  'close'],
 ['There',
  'nothing',
  'VERY',
  'remarkable',
  'Alice',
  'think',
  'VERY',
  'much',
  'way',
  'hear',
  'Rabbit',
  'say',
  'Oh',
  'dear'],
 ['Oh', 'dear'],
 ['shall', 'late'],
 ['thought',
  'afterwards',
  'occurred',
  'ought',
  'wondered',
  'time',
  'seemed',
  'quite',
  '

In [4]:
cols = range(0,maxi)
termsPD = pd.DataFrame(columns=cols)
for row in TermSentences:
    di = {key: None for key in cols}
    for index in range(len(row)):
        di[index] = row[index]
    termsPD = termsPD.append(di, ignore_index=True)
termsPD.to_csv('TermSentences.csv', index = False)
termsPD

,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,Alice,Adventures,Wonderland,Lewis,Carroll,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,CHAPTER,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Down,Rabbit,Hole,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Alice,beginning,get,tired,sitting,sister,bank,nothing,twice,peeped,...,None,None,None,None,None,None,None,None,None,None
4,So,considering,mind,well,could,hot,day,made,feel,sleepy,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,But,sister,sat,still,left,leaning,head,hand,watching,setting,...,None,None,None,None,None,None,None,None,None,None
1699,First,dreamed,little,Alice,tiny,hands,clasped,upon,knee,bright,...,None,None,None,None,None,None,None,None,None,None
1700,The,long,grass,rustled,feet,White,Rabbit,hurried,frightened,Mouse,...,None,None,None,None,None,None,None,None,None,None
1701,So,sat,closed,eyes,half,believed,Wonderland,though,knew,open,...,None,None,None,None,None,None,None,None,None,None


### After converting the data to binary, using lowerBoundMinSupport of .1 and .01 returns 'No rules found'. <br> Using lowerBoundMinSupport of .005 returns 5 rules: the top 4 are some combination of Mock Turtle, and the last is 'Of Course'. Using lowerBoundMinSupport of .001 returns 2156 rules.  Looking at the top 10, a lot of them have to do with how something was said, such as 'loudly said'. The rest have to do with the word the, which should probably be removed as it is a stop word.

#### lowerBoundMinSupport of .005 

![title](weka2.png)

#### lowerBoundMinSupport of .001

![title](weka1.png)

## 2. [50 pts] In the lecture module, the class NeuralNetMLP is a single hidden layer neural network implementation. Make the necessary modifications to upgrade it to a 2 hidden layer network. Run it on the MNIST dataset and report its performance.
(Hint: Raschka, Chapter 12)

I copied the class notebooks class and left it as is. I then make a seconc copy and make adjustments for this question. This is so I can see the side by side performance. My edits improved tha accuracy of the model. The 2 layer algorithm is denoted with a 2 in the name, such as NueralNetMLP2 or nn2. 

In [5]:
warnings.filterwarnings('ignore', 'Solver terminated early.*')
def load_mnist(path, kind='train'):
    from numpy import fromfile, uint8
    import os
    import struct
    
    labels_path = os.path.join(path, '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte' % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = fromfile(lbpath, dtype=uint8)
        with open(images_path, 'rb') as imgpath:
            magic, num, rows, cols = struct.unpack(">IIII",imgpath.read(16))
            images = fromfile(imgpath, dtype=uint8).reshape(len(labels), 784)
            images = ((images / 255.) - .5) * 2
    return images, labels

def get_acc(_y_test, _y_pred):
    return ((np.sum(_y_test == _y_pred)).astype(float) / _y_test.shape[0])

X_train, y_train = load_mnist('mnist/', kind='train')
print(f'Rows= {X_train.shape[0]}, columns= {X_train.shape[1]}')

X_test, y_test = load_mnist('mnist/', kind='t10k')
print(f'Rows= {X_test.shape[0]}, columns= {X_test.shape[1]}')

Rows= 60000, columns= 784
Rows= 10000, columns= 784


In [6]:
class NeuralNetMLP(object):

    def __init__(self, n_hidden=30, epochs=100, eta=0.001, minibatch_size=1, seed=None):
        self.random = np.random.RandomState(seed)  # used to randomize weights
        self.n_hidden = n_hidden  # size of the hidden layer
        self.epochs = epochs  # number of iterations
        self.eta = eta  # learning rate
        self.minibatch_size = minibatch_size  # size of training batch - 1 would not work
    
    @staticmethod
    def onehot(y, n_classes):  # one hot encode the input class y
        onehot = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1.0
        return onehot.T
    
    @staticmethod
    def sigmoid(z):  # Eq 1
        return 1.0 / (1.0 + np.exp(-np.clip(z, -250, 250)))

    def _forward(self, X):  # Eq 2
        z_h = np.dot(X, self.w_h)
        a_h = self.sigmoid(z_h)
        z_out = np.dot(a_h, self.w_out)
        a_out = self.sigmoid(z_out)
        return z_h, a_h, z_out, a_out

    @staticmethod
    def compute_cost(y_enc, output):  # Eq 4
        term1 = -y_enc * (np.log(output))
        term2 = (1.0-y_enc) * np.log(1.0-output)
        cost = np.sum(term1 - term2)
        return cost

    def predict(self, X):
        z_h, a_h, z_out, a_out = self._forward(X)
        y_pred = np.argmax(z_out, axis=1)
        return y_pred

    def fit(self, X_train, y_train, X_valid, y_valid):
        import sys
        n_output = np.unique(y_train).shape[0]  # number of class labels
        n_features = X_train.shape[1]
        self.w_out = self.random.normal(loc=0.0, scale=0.1, size=(self.n_hidden, n_output))
        self.w_h = self.random.normal(loc=0.0, scale=0.1, size=(n_features, self.n_hidden))
        y_train_enc = self.onehot(y_train, n_output)  # one-hot encode original y
        for i in range(self.epochs):
            indices = np.arange(X_train.shape[0])
            for start_idx in range(0, indices.shape[0] - self.minibatch_size + 1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]
                z_h, a_h, z_out, a_out = self._forward(X_train[batch_idx])
                sigmoid_derivative_h = a_h * (1.0-a_h)  # Eq 3
                delta_out = a_out - y_train_enc[batch_idx]  # Eq 5
                delta_h = (np.dot(delta_out, self.w_out.T) * sigmoid_derivative_h)  # Eq 6
                grad_w_out = np.dot(a_h.T, delta_out)  # Eq 7
                grad_w_h = np.dot(X_train[batch_idx].T, delta_h)  # Eq 8
                self.w_out -= self.eta*grad_w_out  # Eq 9
                self.w_h -= self.eta*grad_w_h  # Eq 9
            # Evaluation after each epoch during training
            z_h, a_h, z_out, a_out = self._forward(X_train)
            cost = self.compute_cost(y_enc=y_train_enc, output=a_out)
            y_train_pred = self.predict(X_train)  # monitoring training progress through reclassification
            y_valid_pred = self.predict(X_valid)  # monitoring training progress through validation
            train_acc = ((np.sum(y_train == y_train_pred)).astype(float) / X_train.shape[0])
            valid_acc = ((np.sum(y_valid == y_valid_pred)).astype(float) / X_valid.shape[0])
            sys.stderr.write('\r%d/%d | Cost: %.2f ' '| Train/Valid Acc.: %.2f%%/%.2f%% '%
                (i+1, self.epochs, cost, train_acc*100, valid_acc*100))
            sys.stderr.flush()
        return self
    
class NeuralNetMLP2(object):

    def __init__(self, n_hidden=[30,20], epochs=100, eta=0.001, minibatch_size=1, seed=None):
        self.random = np.random.RandomState(seed)  # used to randomize weights
        self.n_hidden = n_hidden  # size of the hidden layer
        self.epochs = epochs  # number of iterations
        self.eta = eta  # learning rate
        self.minibatch_size = minibatch_size  # size of training batch - 1 would not work
    
    @staticmethod
    def onehot(y, n_classes):  # one hot encode the input class y
        onehot = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1.0
        return onehot.T
    
    @staticmethod
    def sigmoid(z):  # Eq 1
        return 1.0 / (1.0 + np.exp(-np.clip(z, -250, 250)))

    def _forward(self, X):  # Eq 2
        z_h = np.dot(X, self.w_h) 
        a_h = self.sigmoid(z_h) #layer1
        #------------added lines below----------------#
        z_h2 = np.dot(a_h, self.w_h2)
        a_h2 = self.sigmoid(z_h2)
        #----------------edited line below-------------------#
        z_out = np.dot(a_h2, self.w_out)
        #----------------------------------------------------#
        a_out = self.sigmoid(z_out) #out
        return z_h, a_h, z_h2, a_h2, z_out, a_out

    @staticmethod
    def compute_cost(y_enc, output):  # Eq 4
        term1 = -y_enc * (np.log(output))
        term2 = (1.0-y_enc) * np.log(1.0-output)
        cost = np.sum(term1 - term2)
        return cost

    def predict(self, X):
        z_h, a_h, z_h2, a_h2, z_out, a_out = self._forward(X)
        y_pred = np.argmax(z_out, axis=1)
        return y_pred

    def fit(self, X_train, y_train, X_valid, y_valid):
        n_output = np.unique(y_train).shape[0]  # number of class labels
        n_features = X_train.shape[1]
        #----------------edited line below-------------------#
        self.w_out = self.random.normal(loc=0.0, scale=0.1, size=(self.n_hidden[1], n_output))
        self.w_h = self.random.normal(loc=0.0, scale=0.1, size=(n_features, self.n_hidden[0])) 
        #----------------------------------------------------#
        #----------------added line below--------------------#
        self.w_h2 = self.random.normal(loc=0.0,scale=0.1,size=(self.n_hidden[0],self.n_hidden[1]))
        #----------------------------------------------------#
        y_train_enc = self.onehot(y_train, n_output)  # one-hot encode original y
        for i in range(self.epochs):
            indices = np.arange(X_train.shape[0])
            for start_idx in range(0, indices.shape[0] - self.minibatch_size + 1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]
                z_h, a_h, z_h2, a_h2, z_out, a_out = self._forward(X_train[batch_idx])
                sigmoid_derivative_h = a_h * (1.0-a_h)  # Eq 3
                #----------added and edited below------------#
                sigmoid_derivative_h2 = a_h2 * (1.0-a_h2)  # Eq 3
                delta_out = a_out - y_train_enc[batch_idx]  # Eq 5
                delta_h2 = (np.dot(delta_out, self.w_out.T) * sigmoid_derivative_h2)  # Eq 6
                delta_h =  (np.dot(delta_h2, self.w_h2.T) * sigmoid_derivative_h)
                grad_w_out = np.dot(a_h2.T, delta_out)  # Eq 7
                grad_w_h = np.dot(X_train[batch_idx].T, delta_h)  # Eq 8
                grad_w_h2 = np.dot(a_h.T, delta_h2)
                self.w_out -= self.eta*grad_w_out  # Eq 9
                self.w_h -= self.eta*grad_w_h  # Eq 9
                self.w_h2 -= self.eta*grad_w_h2
            # Evaluation after each epoch during training
            z_h, a_h, z_h2, a_h2, z_out, a_out = self._forward(X_train)
            cost = self.compute_cost(y_enc=y_train_enc, output=a_out)
            y_train_pred = self.predict(X_train)  # monitoring training progress through reclassification
            y_valid_pred = self.predict(X_valid)  # monitoring training progress through validation
            train_acc = ((np.sum(y_train == y_train_pred)).astype(float) / X_train.shape[0])
            valid_acc = ((np.sum(y_valid == y_valid_pred)).astype(float) / X_valid.shape[0])
            sys.stderr.write('\r%d/%d | Cost: %.2f ' '| Train/Valid Acc.: %.2f%%/%.2f%% '%
                (i+1, self.epochs, cost, train_acc*100, valid_acc*100))
            sys.stderr.flush()
        return self

In [7]:
nn = NeuralNetMLP(n_hidden=20, epochs=300, eta=0.0005, minibatch_size=100, seed=1)

nn.fit(X_train=X_train[:55000], y_train=y_train[:55000], X_valid=X_train[55000:], y_valid=y_train[55000:]) ;

300/300 | Cost: 15436.03 | Train/Valid Acc.: 96.21%/95.50% 

In [8]:
y_pred = nn.predict(X_test)

print(f'Accuracy= {get_acc(y_test,y_pred)*100:.2f}%')
print(confusion_matrix(y_test,y_pred))

Accuracy= 94.12%
[[ 959    0    1    0    4    1    9    1    5    0]
 [   0 1100    3    2    1    0    5    1   23    0]
 [  14    0  961   13    4    0    9   11   20    0]
 [   2    3   13  955    0   13    1    7   10    6]
 [   0    2    2    1  937    1    5    6    4   24]
 [   8    5    2   39    6  775   13    9   27    8]
 [  13    4    4    0    6   14  903    1   13    0]
 [   3    6   13    7    3    0    0  980    4   12]
 [   3    4    6   12    9    6    5   11  915    3]
 [   4    5    0   17   24    4    1   21    6  927]]


In [9]:
nn2 = NeuralNetMLP2(n_hidden=[30,20], epochs=300, eta=0.0005, minibatch_size=100, seed=1)

nn2.fit(X_train=X_train[:55000], y_train=y_train[:55000], X_valid=X_train[55000:], y_valid=y_train[55000:]) ;

300/300 | Cost: 3648.38 | Train/Valid Acc.: 99.38%/96.70% 

In [10]:
y_pred2 = nn2.predict(X_test)

print(f'Accuracy= {get_acc(y_test,y_pred2)*100:.2f}%')
print(confusion_matrix(y_test,y_pred2))

Accuracy= 95.71%
[[ 961    0    4    2    1    4    4    2    2    0]
 [   0 1116    3    3    0    1    1    2    9    0]
 [   6    4  983   11    2    0    7   10    8    1]
 [   1    3   17  961    0    7    1    5   13    2]
 [   1    2    1    0  946    0    8    4    3   17]
 [   5    2    0   21    3  833    6    3   13    6]
 [   5    2    7    0    9    8  916    0   11    0]
 [   0    7   13    7    4    2    1  985    1    8]
 [   5    2    8    4    7   11    4    3  923    7]
 [   6    6    0    5   17    8    1   11    8  947]]
